In [ ]:
import pathlib

import pandas as pd

import dlsproc.xml
import dlsproc.bundle

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data


## Insiders

> Licitaciones publicadas en los perfiles del contratante ubicados en la Plataforma de Contratación del Sector Público, excluyendo los contratos menores

A file to *cache* the read data

In [ ]:
insiders_output_file = pathlib.Path('insiders.parquet')

The directory for *insiders* data

In [ ]:
insiders_directory = data_directory / 'perfiles_plataforma'
assert insiders_directory.exists()
print(insiders_directory)

/home/manu/dlsproc/data/perfiles_plataforma


The data is read from a file if present, or from the given directory if not

In [ ]:
if insiders_output_file.exists():
    insiders_df = pd.read_parquet(insiders_output_file)
else:
    # data is read from the given directory, and concatenated into a single `pd.DataFrame`...
    insiders_df = dlsproc.bundle.read_zips_directory(insiders_directory, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    insiders_df = dlsproc.xml.flat_df_to_multiindexed_df(insiders_df)
    
    # ...and saved
    insiders_df.to_parquet(insiders_output_file)

In [ ]:
insiders_df.shape

(78944, 148)

In [ ]:
insiders_df.head(3)

id  \
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
zip                                                file name                                     entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom 0      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 1      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 2      https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                  summary  \
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                                                                                                      nan   
                                                                    

In [ ]:
insiders_df.dtypes[:4]

id       nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan                 string
summary  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan                 string
title    nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan                 string
updated  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan    datetime64[ns, UTC]
dtype: object

Number of levels in the *columns*

In [ ]:
insiders_df.columns.nlevels

12

The 3rd column is *updated* 

In [ ]:
insiders_df.columns[3]

('updated',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan')

In order to extract it *as a time series*

In [ ]:
insiders_df.loc[:, insiders_df.columns[3]]

zip                                                  file name                                                      entry
licitacionesPerfilesContratanteCompleto3_202203.zip  licitacionesPerfilesContratanteCompleto3.atom                  0       2022-03-31 20:06:05.604000+00:00
                                                                                                                    1       2022-03-31 19:41:25.548000+00:00
                                                                                                                    2       2022-03-31 19:28:25.157000+00:00
                                                                                                                    3       2022-03-31 19:05:01.018000+00:00
                                                                                                                    4       2022-03-31 19:01:45.736000+00:00
                                                                                             

Without specifying all the levels in the index

In [ ]:
insiders_df.loc[:, 'updated']

,,,nan
,,,nan
,,,nan
,,,nan
,,,nan
,,,nan
,,,nan
,,,nan
,,,nan
,,,nan
,,,nan


In [ ]:
# # insiders_df.xs('updated', axis=1).droplevel(level=list(range(1,11)), axis=1)
# insiders_df.xs('updated', axis=1).droplevel(level=list(range(1,11)), axis=1).droplevel(['zip', 'file name']).iloc[0]

`True` for those columns containing "Name" in their names

In [ ]:
column_name_contains_name = [list(filter(lambda e: (type(e) != float) and ('Name' in e), c)) != [] for c in insiders_df.columns]
print(column_name_contains_name[:15])

[False, False, False, False, False, False, False, False, False, False, False, True, True, False, False]


In [ ]:
# column_name_contains_name = insiders_df.columns.str.contains('Name')
# column_name_contains_name

The corresponding columns in the `pd.DataFrame` are displayed

In [ ]:
insiders_df.loc[:, column_name_contains_name]

ContractFolderStatus  \
                                                                                                                                       LocatedContractingParty   
                                                                                                                                                         Party   
                                                                                                                                                     PartyName   
                                                                                                                                                          Name   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
zip                                                file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom      0      Alcaldía del Ayuntamiento de Fresno de Cantespino   
                                                                                                      1      Consejo de Administración de ICEX España Expor...   
                                                                                                      2             Alcaldia del Ayuntamiento de Palma del Río   
                                                                                                      3      Junta de Gobierno del Ayuntamiento de Palma de...   
                                                                                                      4      Concejalía de Personal y Contratación del Ayun...   
..                                                                                                                                                         ...   
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3_202201... 492    Instituto Balear de Infraestructuras y Servici...   
                                                                                                      493        Servicio Murciano de Salud - Junta de Compras   
                                                                                                      494    Dirección Provincial del Servicio Público de E...   
                                                                                                      495    Junta de Gobierno Local del Ayuntamiento de Ca...   
                                                                                                      496    Alcaldía del Ayuntamiento de Oliva de la Frontera   

                                                                                                                                   \
                                                                                                                    

The two columns below (built from a *path* by means of `dlsproc.xml.assemble_name`) are the most promising ones

In [ ]:
levels_entity_columns = [
    ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'],
    ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'Contact', 'Name']
]
entity_columns = [dlsproc.xml.assemble_name(e) for e in levels_entity_columns]
entity_columns

['ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name',
 'ContractFolderStatus - LocatedContractingParty - Party - Contact - Name']

A function to produce a *full* index path from the upper levels

In [ ]:
def pad_col_levels(df: pd.DataFrame, levels: tuple | list) -> tuple:
    return tuple(list(levels) + ['nan'] * (df.columns.nlevels - len(levels)))

In [ ]:
pad_col_levels(insiders_df, levels_entity_columns[0] )

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan')

The promising columns

In [ ]:
names_df = insiders_df.loc[:, [pad_col_levels(insiders_df, c) for c in levels_entity_columns]]
names_df

ContractFolderStatus  \
                                                                                                                                       LocatedContractingParty   
                                                                                                                                                         Party   
                                                                                                                                                     PartyName   
                                                                                                                                                          Name   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
                                                                                                                                                           nan   
zip                                                file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom      0      Alcaldía del Ayuntamiento de Fresno de Cantespino   
                                                                                                      1      Consejo de Administración de ICEX España Expor...   
                                                                                                      2             Alcaldia del Ayuntamiento de Palma del Río   
                                                                                                      3      Junta de Gobierno del Ayuntamiento de Palma de...   
                                                                                                      4      Concejalía de Personal y Contratación del Ayun...   
..                                                                                                                                                         ...   
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3_202201... 492    Instituto Balear de Infraestructuras y Servici...   
                                                                                                      493        Servicio Murciano de Salud - Junta de Compras   
                                                                                                      494    Dirección Provincial del Servicio Público de E...   
                                                                                                      495    Junta de Gobierno Local del Ayuntamiento de Ca...   
                                                                                                      496    Alcaldía del Ayuntamiento de Oliva de la Frontera   

                                                                                                                                                                
                                                                                        

Are they identical?

In [ ]:
(names_df.iloc[:, 0] == names_df.iloc[:, 1]).all()

True

The depth of the *non-leaf* columns

In [ ]:
# depths = dlsproc.xml.columns_depth(insiders_df)
# depths.head()

The column associated with the *nestedest* tag

In [ ]:
# i_max = depths.index[depths.argmax()]
# nestedest_column = insiders_df.columns[i_max]
# nestedest_column

Non-null entries in that column

In [ ]:
# insiders_df.loc[~insiders_df[nestedest_column].isna()][nestedest_column].values